### Newly derived coefficients<br>
$a = y(\gamma_{i} - 1)$ <br>
$b = z(\gamma_{e} - 1)$ <br>
$c = \frac{m_{p}(\gamma_{i} - 1)(\gamma_{e}-1)}{k_{B}(\gamma - 1)}$ <br>
$C[Pg -> Te] = A = \frac{<Te>}{<Pg>}-\frac{2b<Te><Pg>(R_{h} - <R>)}{(<P_{mag}>^{2}+<Pg>^{2})(a+b<R>)}$ <br>
$C[\rho -> Te] = B = -\frac{<Te>}{<\rho>}$ <br>
$C[B^{2} -> Te] = C = \frac{b<Te><Pg>}{2\mu_{0}(<P_{mag}>^{2}+<Pg>^{2})(a+b<R>)}$

In [7]:
# some constants
cl = 29979245800.0 # speed of light in cgs
mp = 1.67262171e-24
kb = 1.3806505e-16
mu = 1.

In [8]:
# constants used for the functions
y = 1.
z = 3.
gamma = 4/3
gamma_i = 4/3
gamma_e = 7/3
a = y*(gamma_i - 1)
b = z*(gamma_e - 1)
c = mp*(gamma_e - 1)*(gamma_i - 1)/(kb*(gamma - 1))

In [9]:
def CPg(avgTe, avgPg, avgPmag, avgR, Rh):
    term1 = avgTe/avgPg
    term2 = 2*b*avgTe*avgPg*(Rh-avgR)/((avgPmag**2+avgPg**2)*(a+b*avgR))
    return term1-term2

In [10]:
def Crho(avgTe, avgrho):
    return -avgTe/avgrho

In [11]:
def Cbsq(avgTe, avgPg, avgPmag, avgR):
    nume = b*avgTe*avgPg
    denom = 2*mu*(avgPmag**2+avgPg**2)*(a+b*avgR)
    return nume/denom

## Test:
<h3>Check if coeffs are working properly: Use Te's long equation to get a list of Te values and calculate varTe</h3> <br>
var(Te) = $(\delta Te)^{2}$ = $A^{2}$var(Pg) + $B^{2}$var($\rho$) + $C^{2}$var(bsq) + 2$AB$cov(Pg, $\rho$) + 2$AC$ cov(Pg, bsq) + 2$BC$ cov($\rho$, bsq)

In [21]:
# cross terms
macov_rhoPg = np.load("ma_avgRHOPg.npy", allow_pickle=True).item()['rhoPg'] - maPg_avg * maRHO_avg
macov_PgPmag = np.load("ma_avgPgPmag.npy", allow_pickle=True).item()['PgPmag']
macov_rhoPmag = np.load("ma_avgRHOPmag.npy", allow_pickle=True).item()['rhoPmag']

sacov_rhoPg = np.load("sa_avgRHOPg.npy", allow_pickle=True).item()['rhoPg']- saPg_avg * saRHO_avg
sacov_PgPmag = np.load("sa_avgPgPmag.npy", allow_pickle=True).item()['PgPmag']
sacov_rhoPmag = np.load("sa_avgRHOPmag.npy", allow_pickle=True).item()['rhoPmag']

In [22]:
# change cross terms
macov_Pgbsq = 2*mu*macov_PgPmag - maPg_avg*mabsq_avg
macov_rhobsq = 2*mu*macov_rhoPmag - maRHO_avg*mabsq_avg

sacov_Pgbsq = 2*mu*sacov_PgPmag - saPg_avg*sabsq_avg
sacov_rhobsq = 2*mu*sacov_rhoPmag - saRHO_avg*sabsq_avg

In [23]:
# R
def R(beta, Rh=1., Rl=1.):
    return Rl/(1+beta**2)+Rh*beta**2/(1+beta**2)

In [24]:
def Te(beta, Pg, rho, Rh=1., Rl=1., gamma=4/3):
    if gamma == 4/3:
        y = 1
        z = 3
        gamma_i = 4/3
        gamma_e = 7/3
    if gamma == 13/9:
        y = 1
        z = 2
        gamma_i = 13/9
        gamma_e = 133/36
    if gamma == 5/3:
        y = 1
        z = 1
        gamma_i = 5/3
        gamma_e = 5/3
    u = Pg/(gamma-1)
    r = R(beta, Rh, Rl)
    numerator = mp*cl**2*u*(gamma_e-1)*(gamma_i-1)
    denom = kb*rho*(y*(gamma_i-1)+r*z*(gamma_e-1))

    electron_T = numerator/ denom
    return electron_T

<h5>test: generate a list of different random number grids and then calculate their variance by 1) direct calculations from Te OR 2) summation of variance terms </h5>

1) direct calculations from Te

In [25]:
# test: read in coeffs
# MAD, Rh=1
ma_CPg1 = np.load('ma_CPg1.npy')
ma_Crho1 = np.load('ma_Crho1.npy')
ma_Cbsq1 = np.load('ma_Cbsq1.npy')

In [26]:
ma_CPg1.shape

(288, 128, 128)

In [27]:
pg_lst = []
rho_lst = []
bsq_lst = []
beta_lst = []
Te_calc_lst = []
for i in range(0, 5):
    pg = np.random.random(ma_CPg1.shape)
    pg_lst.append(pg)
    rho = np.random.random(ma_Crho1.shape)
    rho_lst.append(rho)
    bsq = np.random.random(ma_Cbsq1.shape)
    bsq_lst.append(bsq)
    beta = np.random.random(ma_CPg1.shape)
    beta_lst.append(beta)
    Te_calc = Te(beta, pg, rho)
    Te_calc_lst.append(Te_calc)

In [28]:
pg_values = np.array(pg_lst)
rho_values = np.array(rho_lst)
bsq_values = np.array(bsq_lst)
beta_values = np.array(beta_lst)
Te_calc_values = np.array(Te_calc_lst)

In [29]:
var_pg = np.var(pg_values, axis=0)
var_pg.shape

(288, 128, 128)

In [30]:
var_rho = np.var(rho_values, axis=0)
var_bsq = np.var(bsq_values, axis=0)
var_beta = np.var(beta_values, axis=0)
varTe_calc = np.var(Te_calc_values, axis=0)

2) summation of variance terms

In [31]:
def cov(arr1, arr2):
    arr1_mean = np.mean(arr1, axis=0)
    arr2_mean = np.mean(arr2, axis=0)
    products = []
    for i in range(0, 5):
        prod = arr1[i] * arr2[i]
        products.append(prod)
    product_mean = np.mean(np.array(products), axis=0)
    return product_mean-arr1_mean*arr2_mean

In [32]:
cov_pgrho = cov(pg_values, rho_values)
cov_pgrho.shape

(288, 128, 128)

In [33]:
cov_pgbsq = cov(pg_values, bsq_values)
cov_rhobsq = cov(rho_values, bsq_values)

In [52]:
varTe_coeff = np.square(ma_CPg1)*var_pg + np.square(ma_Crho1)*var_rho + np.square(ma_Cbsq1) * var_bsq + \
            2*ma_CPg1*ma_Crho1*cov_pgrho + 2*ma_CPg1*ma_Cbsq1*cov_pgbsq + 2*ma_Crho1*ma_Cbsq1 * cov_rhobsq

In [56]:
varTe_coeff

array([[7.56779355e+26, 9.13187591e+26, 9.11147929e+26, ...,
        8.15204399e+26, 1.05567007e+27, 1.10641422e+27],
       [8.51881981e+26, 1.37074007e+27, 8.07030692e+26, ...,
        3.81984020e+26, 5.18733877e+26, 1.64341092e+27],
       [1.42603553e+27, 1.13035896e+27, 2.40888270e+26, ...,
        1.78108909e+27, 2.63258944e+26, 9.99446084e+26],
       ...,
       [1.54860761e+27, 1.15190702e+27, 8.55709455e+26, ...,
        1.23902496e+27, 3.34467932e+26, 6.91819297e+26],
       [1.08821336e+27, 2.99705971e+26, 4.96709007e+26, ...,
        1.11886690e+27, 7.91749452e+26, 1.73237315e+27],
       [4.17886291e+26, 1.80755705e+27, 4.70202532e+27, ...,
        6.35930457e+26, 1.31806152e+27, 4.18474058e+27]])

In [63]:
varTe_calc

array([[[1.16541606e+26, 5.16901152e+25, 2.13976051e+25, ...,
         6.91665325e+23, 9.91488801e+24, 1.99206437e+26],
        [7.51242720e+24, 1.01690118e+25, 5.28743938e+25, ...,
         6.70917227e+24, 4.09541150e+25, 7.31534340e+25],
        [2.31637835e+26, 4.97975489e+26, 2.89393315e+25, ...,
         4.37156456e+24, 2.50741422e+23, 1.26746784e+26],
        ...,
        [3.12866520e+25, 3.60315038e+25, 1.75869353e+24, ...,
         6.55937167e+25, 2.35343323e+24, 1.49409244e+27],
        [9.66492305e+26, 7.42509029e+25, 9.14634846e+24, ...,
         3.51897184e+24, 4.84290530e+24, 5.01438094e+25],
        [2.80328568e+24, 8.49370067e+26, 3.02293084e+25, ...,
         1.05310345e+26, 4.38121730e+24, 1.25025054e+26]],

       [[2.31587799e+24, 5.78941049e+26, 1.06100395e+23, ...,
         1.95136841e+25, 8.06401407e+23, 2.97582891e+24],
        [1.78807952e+25, 5.20551700e+25, 4.71850973e+26, ...,
         5.55046909e+23, 3.94836136e+26, 1.63502802e+26],
        [2.74703938e+28, 

In [62]:
var_bsq

array([[[0.03630905, 0.03927567, 0.01753879, ..., 0.06408724,
         0.04435208, 0.10044517],
        [0.03071065, 0.04741271, 0.06384511, ..., 0.08975612,
         0.06669829, 0.07849137],
        [0.0992089 , 0.08228761, 0.07194008, ..., 0.14804604,
         0.07037098, 0.10068701],
        ...,
        [0.06609619, 0.00864754, 0.14593886, ..., 0.01907966,
         0.08663158, 0.03786457],
        [0.02806297, 0.04633568, 0.03797351, ..., 0.11510928,
         0.06398439, 0.08716834],
        [0.05132408, 0.06401046, 0.08182374, ..., 0.02609758,
         0.04006474, 0.06575918]],

       [[0.05110258, 0.03014889, 0.06079196, ..., 0.06062891,
         0.11658748, 0.0272259 ],
        [0.07374927, 0.10102529, 0.03490835, ..., 0.05387479,
         0.14348988, 0.04836635],
        [0.08095779, 0.04279491, 0.0780639 , ..., 0.10591892,
         0.06528233, 0.05490951],
        ...,
        [0.00447805, 0.08693298, 0.04193807, ..., 0.11311389,
         0.04382613, 0.09560696],
        [0.0

Equation: var(Te) = $(\delta Te)^{2}$ = $A^{2}$var(Pg) + $B^{2}$var($\rho$) + $C^{2}$var(bsq) + 2$AB$cov(Pg, $\rho$) + 2$AC$ cov(Pg, bsq) + 2$BC$ cov($\rho$, bsq) <br>

Order of magnitude (following the grid's order; upper and lower limits taken from [10%, 90%]): <br>
coefficients: <br>
C[Pg->Te]: $10^{13}$ -> $10^{16}$ <br>
C[$\rho$->Te]: $-10^{11}$ -> $-10^{15}$ <br>
C[bsq->Te]: $10^{9}$ -> $10^{15}$ <br>

generated, independent values: <br>
var_Pg: $10^{-2}$ -> $10^{-1}$ <br> 
var_rho: $10^{-2}$ -> $10^{-1}$ <br> 
var_bsq: $10^{-2}$ -> $10^{-1}$ <br> 
=> In this case, Te is expected to be $10^{23}$ to $10^{26}$ while Te (coeff) ranges from $10^{23}$ to $10^{36}$. The order of mag is off by (up to) $10^{10}$. It's caused by the fact that the randomly generated Pg, rho, and bsq values are constrained to be between 0 and 1. They do not follow the real trend of Pg, rho, and bsq values.

real values: <br>
var_Te: $10^{22}$ -> $10^{15}$ <br>
var_Pg: $10^{-4}$ -> $10^{-15}$ <br>
var_rho: $10^{-2}$ -> $10^{-11}$ <br>
var_bsq: $10^{-2}$ -> $10^{-17}$ <br>
cov_PgRHO: $10^{-3}$ -> $10^{-14}$ <br>
`The transition from positive to negative values and the extremely small values contained in this process mess up with the np.percentile function, so the following ranges are estimations:` <br>
cov_Pgbsq:  $10^{-3}$->$-10^{-13}$<br>
cov_RHObsq: $10^{-3}$->$-10^{-11}$ <br>
Following the equation, at the beginning part of the grid, RHS of the quation yields $10^{26}\times 10^{-4} + 10^{22}\times 10^{-2} + 10^{18}\times 10^{-2} + 2\times 10^{13}\times (-10^{11})\times 10^{-3} + 2\times 10^{13} \times 10^{9} \times 10^{-3} + 2\times (-10^{11}) \times 10^{9} \times 10^{-3} \approx 10^{22}$; <br> at the end part of the grid, RHS of the equation yields $10^{32}\times 10^{-15} + 10^{30}\times 10^{-11} + 10^{30}\times 10^{-17} + 2\times 10^{16} \times (-10^{15})\times 10^{-14} + 2\times 10^{16} \times 10^{15} \times (-10^{-13}) + 2\times (-10^{15})\times 10^{15} \times 10^{-11} \approx 10^{18}$ <br> The lower limit is close to 40% percentile of varTe, which may be caused by the estimations of covariances

If they are: Do the var(Te) [two versions: 1) calculated from error propagation and 2) directly calculated] VS theta ([0, pi]) plots at different radius (i.e. different index slices of that (288, 128, 128) array). Say, radius r=2, then choose the slice varTe[2, :, 0] so that you got a 1D var(Te) at radius r=2 along theta